## Extract labels of the video with Google Cloud Video Intelligence

In [ ]:
import os
import pandas as pd
from google.cloud import videointelligence

# Set Google Cloud Video Intelligence API Client
json_path = './path/to/json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_path

# Use Google Cloud Video Intelligence to annotate a video
def videoAnnotation(videoName):
    # Call the API 
    client = videointelligence.VideoIntelligenceServiceClient() 
    job = client.annotate_video(
    input_uri='gs://humour_video/{}'.format(videoName),
    features=[videointelligence.enums.Feature.LABEL_DETECTION])
    
    print('\nStart annotation of {}'.format(videoName))
    result = job.result(timeout=90)
    print('Video {} has been annoted'.format(videoName))
    return result

# Get annotation from Google Cloud Video Intelligence and save it in csv form
# The algo will be performed on all the video, unless shot is True
def annotation_to_CSV(videoName, result, shot=False):
    
    print('Converting {} annotation to CSV'.format(videoName))
    
    df = pd.DataFrame()
        
    # Get the result of the annotation per segment (here per video)
    segment_labels = result.annotation_results[0].segment_label_annotations
    
    if shot:
        segment_labels = result.annotation_results[0].shot_label_annotations
    
    # Enumerate through the class to get all the labels and add it to a dataframe
    for i, segment_label in enumerate(segment_labels):
        
        video = {}
        video['Name'] = videoName
        video['VideoLabel'] = segment_label.entity.description

        i = 0
        for category_entity in segment_label.category_entities:
            if i == 0 :
                video['category'] = category_entity.description
                i+=1
            else:
                video['category'+str(i)] = category_entity.description
                i+=1

        for i, segment in enumerate(segment_label.segments):
            video['start_time'] = (segment.segment.start_time_offset.seconds +
                          segment.segment.start_time_offset.nanos / 1e9)
            video['end_time'] = (segment.segment.end_time_offset.seconds +
                        segment.segment.end_time_offset.nanos / 1e9)
            video['confidence'] = segment.confidence


        df = df.append(video, ignore_index=True)
    
    if df.empty:
        print('Video {} df was empty / the video was not save in a csv'.format(videoName))
    
    else:
        if shot:
            df.to_csv('results/shot/{}.csv'.format(videoName[0:-4]), index=False)
        else:
            df.to_csv('results/segment/{}.csv'.format(videoName[0:-4]), index=False)
        
        print('Video {} has been saved into a csv'.format(videoName))
    
    return df



### Code exemple

In [ ]:
# """ 
# Code exemple to analyse video. Get:
# - 1 csv per video (saved by the function)
# - 1 csv for all videos in the videos list (save by this code)
# """

video = '1Neutral_officework.mp4' # set the name of the video that want to analyse

result = videoAnnotation(video)
df_video = annotation_to_CSV(video, result, shot=False) # get a df of the annotation

### Analysis

In [ ]:
from os import listdir, path
import pandas as pd

# Settings
video_path = './path/to/file' # videos in this folder are already loaded in a bucket in google cloud
analysis_path = './path/to/file'
videos = listdir(video_path)
all_annotation = pd.DataFrame()
overwrite = True # If true, return the analysis and save. If Fasle, skip the video 

for video in videos:
    
    if not overwrite and path.exists(analysis_path + video + '.csv'):
        print('Video {} already exist / video was not computed'.format(video))
        continue
    
    annotation = videoAnnotation(video)
    df_video = annotation_to_CSV(video, annotation, shot=True) # get a df of the annotation
    all_annotation = all_annotation.append(df_video, sort=True) # Append this new df to all previous df annotation

all_annotation.to_csv(analysis_path + 'all_data_n{}.csv'.format(len(videos)), index=False) # Save all annotation in one csv